In [9]:
import pyaudio
import wave
import whisper
import time # Pour la durée d'enregistrement

# --- Votre fonction de transcription Whisper ---
def whisper_transcription(file_path, model_name="tiny"):
    """
    Transcrit un fichier audio en utilisant OpenAI Whisper.

    Args:
        file_path (str): Chemin vers le fichier audio.
        model_name (str): Nom du modèle Whisper à utiliser (ex: "tiny", "base", "small", "medium", "large").
                          Les modèles plus grands sont plus précis mais plus lents et gourmands en ressources.
    Returns:
        str: Le texte transcrit.
    """
    try:
        model = whisper.load_model(model_name)

        # Charger l'audio et le préparer pour Whisper
        audio = whisper.load_audio(file_path)
        audio = whisper.pad_or_trim(audio) # S'assure que l'audio a la bonne longueur (30s)

        # Calculer le spectrogramme log-Mel
        # model.dims.n_mels donne le nombre de bandes Mel attendu par le modèle
        mel = whisper.log_mel_spectrogram(audio, n_mels=model.dims.n_mels).to(model.device)

        # Définir les options de décodage
        # fp16=False peut être nécessaire si vous n'avez pas de GPU compatible ou si vous rencontrez des erreurs.
        options = whisper.DecodingOptions(fp16=False if model.device.type == 'cpu' else True) # fp16=False sur CPU

        # Décoder l'audio
        result = whisper.decode(model, mel, options)

        return result.text
    except Exception as e:
        return f"Erreur lors de la transcription : {e}"

# --- Fonction pour enregistrer l'audio ---
def record_audio(filename="recorded_audio.wav", record_seconds=5, rate=16000, chunk_size=1024, channels=1, audio_format=pyaudio.paInt16):
    """
    Enregistre l'audio du microphone et le sauvegarde dans un fichier WAV.

    Args:
        filename (str): Nom du fichier WAV de sortie.
        record_seconds (int): Durée de l'enregistrement en secondes.
        rate (int): Taux d'échantillonnage (Hz). Whisper préfère 16000 Hz.
        chunk_size (int): Nombre de trames par buffer.
        channels (int): Nombre de canaux audio (1 pour mono, 2 pour stéréo). Mono est suffisant pour la parole.
        audio_format (pyaudio.paFormat): Format des échantillons audio.
    """
    p = pyaudio.PyAudio()

    print(f"Enregistrement audio pendant {record_seconds} secondes...")

    stream = p.open(format=audio_format,
                    channels=channels,
                    rate=rate,
                    input=True,
                    frames_per_buffer=chunk_size)

    frames = []

    for i in range(0, int(rate / chunk_size * record_seconds)):
        try:
            data = stream.read(chunk_size)
            frames.append(data)
        except IOError as e:
            if e.errno == pyaudio.paInputOverflowed:
                print("Avertissement : Input overflowed. Des données audio ont pu être perdues.")
                # On peut choisir de continuer ou d'ignorer ce chunk
            else:
                raise # Renvoyer d'autres IOErrors

    print("Enregistrement terminé.")

    # Arrêter et fermer le flux
    stream.stop_stream()
    stream.close()
    p.terminate()

    # Sauvegarder les données audio dans un fichier WAV
    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(audio_format))
    wf.setframerate(rate)
    wf.writeframes(b''.join(frames))
    wf.close()
    print(f"Audio sauvegardé sous : {filename}")
    return filename

# --- Programme principal ---
if __name__ == "__main__":
    output_filename = "mon_enregistrement.wav"
    duration_seconds = 10  # Vous pouvez changer cette durée ou la demander à l'utilisateur

    # Étape 1: Enregistrer l'audio
    try:
        recorded_file = record_audio(
            filename=output_filename,
            record_seconds=duration_seconds,
            rate=16000,       # Whisper est optimisé pour 16kHz
            channels=1        # Mono est généralement préférable pour la transcription vocale
        )
    except Exception as e:
        print(f"Une erreur est survenue lors de l'enregistrement : {e}")
        print("Vérifiez que vous avez un microphone connecté et que les permissions sont accordées.")
        exit()

    # Étape 2: Transcrire l'audio enregistré
    print("\nLancement de la transcription...")
    # Vous pouvez choisir un autre modèle si besoin : "base", "small", "medium", "large"
    # "tiny" est rapide mais moins précis. "base" est un bon compromis.
    model_to_use = "base" # Essayez "tiny" si "base" est trop lent, ou "small" pour plus de précision
    transcription = whisper_transcription(recorded_file, model_name=model_to_use)

    print("\n--- Transcription ---")
    print(transcription)
    print("---------------------")

    # Optionnel: Supprimer le fichier audio après transcription
    # import os
    # try:
    #     os.remove(recorded_file)
    #     print(f"Fichier {recorded_file} supprimé.")
    # except OSError as e:
    #     print(f"Erreur lors de la suppression du fichier {recorded_file}: {e}")

Enregistrement audio pendant 10 secondes...
Enregistrement terminé.
Audio sauvegardé sous : mon_enregistrement.wav

Lancement de la transcription...

--- Transcription ---
Thank you.
---------------------


In [ ]:
PATH_TO_ACCENT_MODEL = "../notebooks/Whisper/modele_logistique.pkl"  # Exemple: à remplacer
PATH_TO_LABEL_ENCODER = "../notebooks/Whisper/label_encoder.pkl"

In [ ]:
import streamlit as st
import pyaudio
import wave
import whisper
import time
import os
import tempfile # Pour gérer les fichiers temporaires

# --- Votre fonction de transcription Whisper (inchangée) ---
def whisper_transcription(file_path, model_name="tiny"):
    try:
        # Afficher un message pendant le chargement du modèle si ce n'est pas déjà fait
        # (Whisper met en cache les modèles téléchargés)
        with st.spinner(f"Chargement du modèle Whisper '{model_name}'... (peut prendre du temps la première fois)"):
            model = whisper.load_model(model_name)

        # st.info(f"Modèle '{model_name}' chargé. Chargement de l'audio...")
        audio = whisper.load_audio(file_path)
        audio = whisper.pad_or_trim(audio)

        # st.info("Calcul du spectrogramme log-Mel...")
        mel = whisper.log_mel_spectrogram(audio, n_mels=model.dims.n_mels).to(model.device)

        # st.info("Décodage de l'audio...")
        # fp16=False peut être nécessaire si vous n'avez pas de GPU compatible.
        options = whisper.DecodingOptions(fp16=False if model.device.type == 'cpu' else True, language="fr") # Spécifier la langue peut aider
        result = whisper.decode(model, mel, options)

        return result.text
    except Exception as e:
        st.error(f"Erreur lors de la transcription Whisper : {e}")
        return None

# --- Fonction pour enregistrer l'audio (légèrement modifiée pour Streamlit) ---
def record_audio(filename_prefix="temp_audio_streamlit", record_seconds=5, rate=16000, chunk_size=1024, channels=1, audio_format=pyaudio.paInt16):
    p = pyaudio.PyAudio()

    # Crée un fichier temporaire pour l'enregistrement
    # delete=False car Whisper doit pouvoir ouvrir le fichier par son nom.
    # Nous le supprimerons manuellement plus tard.
    temp_file = tempfile.NamedTemporaryFile(prefix=filename_prefix, suffix=".wav", delete=False)
    output_filename = temp_file.name
    temp_file.close() # Fermer le handle pour que wave puisse l'ouvrir

    # Message dans Streamlit
    status_placeholder = st.empty()
    status_placeholder.info(f"🔴 Enregistrement en cours pendant {record_seconds} secondes... Parlez maintenant !")

    stream = p.open(format=audio_format,
                    channels=channels,
                    rate=rate,
                    input=True,
                    frames_per_buffer=chunk_size)

    frames = []
    for i in range(0, int(rate / chunk_size * record_seconds)):
        try:
            data = stream.read(chunk_size)
            frames.append(data)
        except IOError as e:
            if e.errno == pyaudio.paInputOverflowed:
                print("Avertissement : Input overflowed. Des données audio ont pu être perdues.")
            else:
                raise

    status_placeholder.success(f"✅ Enregistrement terminé. Sauvegarde en cours...")

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(output_filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(audio_format))
    wf.setframerate(rate)
    wf.writeframes(b''.join(frames))
    wf.close()

    status_placeholder.empty() # Effacer le message de statut
    return output_filename

# --- Interface Streamlit ---
st.set_page_config(page_title="Audio Transcriber", layout="wide")
st.title("🎤 Enregistreur et Transcripteur Audio avec OpenAI Whisper")
st.markdown("Enregistrez un court message audio et obtenez sa transcription.")

# Initialiser l'état de session pour stocker le chemin du fichier et la transcription
if "audio_file_path" not in st.session_state:
    st.session_state.audio_file_path = None
if "transcription_text" not in st.session_state:
    st.session_state.transcription_text = ""

# Options pour l'utilisateur
col1, col2 = st.columns(2)
with col1:
    duration_seconds = st.slider(
        "Durée de l'enregistrement (secondes) :",
        min_value=3,
        max_value=30, # Whisper fonctionne mieux sur des segments courts (max 30s par défaut)
        value=5,
        step=1
    )
with col2:
    model_options = ["tiny", "base", "small", "medium", "large"]
    # "large-v2" ou "large-v3" si vous avez la dernière version de whisper
    # model_options = ["tiny", "base", "small", "medium", "large", "large-v2", "large-v3"]
    selected_model = st.selectbox(
        "Choisissez le modèle Whisper :",
        options=model_options,
        index=model_options.index("base"), # "base" est un bon compromis
        help="Les modèles plus grands sont plus précis mais plus lents et gourmands en ressources."
    )

# Bouton d'enregistrement et de transcription
if st.button("🎙️ Démarrer l'enregistrement et Transcrire", type="primary", use_container_width=True):
    # Nettoyer l'ancien fichier audio s'il existe
    if st.session_state.audio_file_path and os.path.exists(st.session_state.audio_file_path):
        try:
            os.remove(st.session_state.audio_file_path)
            # st.info(f"Ancien fichier {st.session_state.audio_file_path} supprimé.")
        except Exception as e:
            st.warning(f"Impossible de supprimer l'ancien fichier audio : {e}")
    st.session_state.audio_file_path = None
    st.session_state.transcription_text = ""

    try:
        # Étape 1: Enregistrer l'audio
        with st.spinner("Préparation de l'enregistrement..."):
            recorded_file_path = record_audio(
                record_seconds=duration_seconds,
                rate=16000,
                channels=1
            )
        st.session_state.audio_file_path = recorded_file_path
        st.success(f"Audio enregistré : {os.path.basename(recorded_file_path)}")

        # Étape 2: Transcrire l'audio enregistré
        with st.spinner(f"Transcription avec le modèle '{selected_model}' en cours... Ceci peut prendre un moment."):
            transcription = whisper_transcription(st.session_state.audio_file_path, model_name=selected_model)

        if transcription:
            st.session_state.transcription_text = transcription
        else:
            st.session_state.transcription_text = "La transcription a échoué ou n'a rien retourné."

    except Exception as e:
        st.error(f"Une erreur globale est survenue : {e}")
        st.error("Vérifiez que vous avez un microphone connecté et que les permissions sont accordées. Consultez la console pour plus de détails.")
        # Nettoyer en cas d'erreur
        if st.session_state.audio_file_path and os.path.exists(st.session_state.audio_file_path):
            try:
                os.remove(st.session_state.audio_file_path)
            except: pass # Ignorer les erreurs de suppression ici
        st.session_state.audio_file_path = None
        st.session_state.transcription_text = ""


# Afficher le lecteur audio et la transcription s'ils existent
if st.session_state.audio_file_path and os.path.exists(st.session_state.audio_file_path):
    st.subheader("🎧 Audio Enregistré :")
    try:
        with open(st.session_state.audio_file_path, 'rb') as audio_file:
            audio_bytes = audio_file.read()
        st.audio(audio_bytes, format='audio/wav')
    except Exception as e:
        st.warning(f"Impossible de charger le lecteur audio : {e}. Le fichier est peut-être corrompu ou a été supprimé.")

if st.session_state.transcription_text:
    st.subheader("📝 Transcription :")
    st.text_area("Texte transcrit", st.session_state.transcription_text, height=200,
                 help="Vous pouvez copier ce texte.")

st.markdown("---")
st.caption("Application développée avec Streamlit et OpenAI Whisper.")

# Note sur le nettoyage :
# Les fichiers temporaires créés avec delete=False ne sont pas automatiquement supprimés
# à la fermeture du handle. Nous les supprimons au début d'un nouvel enregistrement.
# Le dernier fichier créé pourrait rester jusqu'à ce que le système d'exploitation nettoie /tmp
# ou que l'application soit redémarrée et qu'un nouvel enregistrement soit fait.
# Pour une application en production, une stratégie de nettoyage plus robuste serait nécessaire.